In [1]:
import tensorflow
from tensorflow import keras
from keras import layers, optimizers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

2025-04-19 23:04:19.921674: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import kaggle
import kagglehub

/Users/chuck/python-projects/kaggle/kidney_disease/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path = kagglehub.dataset_download("amanik000/kidney-disease-dataset")

In [4]:
df = pd.read_csv(f"{path}/kidney_disease_dataset.csv")

In [5]:
df.head()

,Age of the patient,Blood pressure (mm/Hg),Specific gravity of urine,Albumin in urine,Sugar in urine,Red blood cells in urine,Pus cells in urine,Pus cell clumps in urine,Bacteria in urine,Random blood glucose level (mg/dl),...,Smoking status,Body Mass Index (BMI),Physical activity level,Duration of diabetes mellitus (years),Duration of hypertension (years),Cystatin C level,Urinary sediment microscopy results,C-reactive protein (CRP) level,Interleukin-6 (IL-6) level,Target
0,54,167,1.023,1,4,normal,abnormal,not present,not present,96,...,yes,25.3,low,4,16,0.67,normal,4.88,10.23,No_Disease
1,42,127,1.023,3,2,normal,normal,not present,present,73,...,no,20.6,moderate,3,13,0.55,abnormal,4.49,13.11,Low_Risk
2,38,148,1.016,0,0,abnormal,normal,not present,not present,77,...,no,38.4,high,11,23,2.37,abnormal,4.57,13.27,No_Disease
3,7,98,1.017,4,0,abnormal,normal,not present,present,225,...,no,24.7,high,24,3,2.54,abnormal,8.57,12.36,No_Disease
4,67,174,1.015,1,1,normal,abnormal,not present,not present,376,...,yes,17.6,high,22,24,1.90,normal,6.75,1.46,No_Disease


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20538 entries, 0 to 20537
Data columns (total 43 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Age of the patient                           20538 non-null  int64  
 1   Blood pressure (mm/Hg)                       20538 non-null  int64  
 2   Specific gravity of urine                    20538 non-null  float64
 3   Albumin in urine                             20538 non-null  int64  
 4   Sugar in urine                               20538 non-null  int64  
 5   Red blood cells in urine                     20538 non-null  object 
 6   Pus cells in urine                           20538 non-null  object 
 7   Pus cell clumps in urine                     20538 non-null  object 
 8   Bacteria in urine                            20538 non-null  object 
 9   Random blood glucose level (mg/dl)           20538 non-null  int64  
 10

In [7]:
df.isna().sum()

Age of the patient                             0
Blood pressure (mm/Hg)                         0
Specific gravity of urine                      0
Albumin in urine                               0
Sugar in urine                                 0
Red blood cells in urine                       0
Pus cells in urine                             0
Pus cell clumps in urine                       0
Bacteria in urine                              0
Random blood glucose level (mg/dl)             0
Blood urea (mg/dl)                             0
Serum creatinine (mg/dl)                       0
Sodium level (mEq/L)                           0
Potassium level (mEq/L)                        0
Hemoglobin level (gms)                         0
Packed cell volume (%)                         0
White blood cell count (cells/cumm)            0
Red blood cell count (millions/cumm)           0
Hypertension (yes/no)                          0
Diabetes mellitus (yes/no)                     0
Coronary artery dise

In [8]:
import sklearn
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [9]:
x_df = df.drop(columns=["Target"])
y_df = df["Target"]

In [10]:
y_df.unique()

array(['No_Disease', 'Low_Risk', 'Moderate_Risk', 'Severe_Disease',
       'High_Risk'], dtype=object)

In [11]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y_df)

In [12]:
y_encoded.shape

(20538,)

In [13]:
y_encoded[0]

3

In [14]:
y_df[0]

'No_Disease'

In [15]:
y_encoded_df = pd.DataFrame(y_encoded)

In [16]:
y_encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20538 entries, 0 to 20537
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       20538 non-null  int64
dtypes: int64(1)
memory usage: 160.6 KB


In [17]:
y_encoded_df.head()

,0
0,3
1,1
2,3
3,3
4,3


In [18]:
cat_cols = x_df.select_dtypes(include=["object"]).columns

In [19]:
x_df = pd.get_dummies(x_df, columns=cat_cols, drop_first=True)

In [20]:
x_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20538 entries, 0 to 20537
Data columns (total 43 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Age of the patient                            20538 non-null  int64  
 1   Blood pressure (mm/Hg)                        20538 non-null  int64  
 2   Specific gravity of urine                     20538 non-null  float64
 3   Albumin in urine                              20538 non-null  int64  
 4   Sugar in urine                                20538 non-null  int64  
 5   Random blood glucose level (mg/dl)            20538 non-null  int64  
 6   Blood urea (mg/dl)                            20538 non-null  float64
 7   Serum creatinine (mg/dl)                      20538 non-null  float64
 8   Sodium level (mEq/L)                          20538 non-null  float64
 9   Potassium level (mEq/L)                       20538 non-null 

In [21]:
x_df.head()

,Age of the patient,Blood pressure (mm/Hg),Specific gravity of urine,Albumin in urine,Sugar in urine,Random blood glucose level (mg/dl),Blood urea (mg/dl),Serum creatinine (mg/dl),Sodium level (mEq/L),Potassium level (mEq/L),...,Diabetes mellitus (yes/no)_yes,Coronary artery disease (yes/no)_yes,Appetite (good/poor)_poor,Pedal edema (yes/no)_yes,Anemia (yes/no)_yes,Family history of chronic kidney disease_yes,Smoking status_yes,Physical activity level_low,Physical activity level_moderate,Urinary sediment microscopy results_normal
0,54,167,1.023,1,4,96,169.101369,7.55,146.068410,6.272576,...,True,False,False,False,False,False,True,True,False,True
1,42,127,1.023,3,2,73,183.223479,13.37,123.501427,5.611303,...,True,False,False,True,True,True,False,False,True,False
2,38,148,1.016,0,0,77,193.141665,9.49,149.456527,3.965957,...,False,True,False,True,False,False,False,False,False,False
3,7,98,1.017,4,0,225,125.939396,10.98,131.758843,4.980997,...,False,True,False,False,True,False,False,False,False,False
4,67,174,1.015,1,1,376,197.188600,3.01,120.912465,4.097602,...,False,False,False,True,True,False,True,False,False,True


In [22]:
x_train, x_val, y_train, y_val = train_test_split(
    x_df, y_encoded_df, test_size=0.2, random_state=42)

In [23]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

In [24]:
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(16430, 43) (16430, 1) (4108, 43) (4108, 1)


In [25]:
y_train = np.array(y_train).reshape((16430,)).astype(int)
y_train.shape

(16430,)

In [26]:
y_val = np.array(y_val).reshape((4108,)).astype(int)
y_val.shape

(4108,)

In [38]:
optimizer = optimizers.Adam(learning_rate=0.001)
model = keras.Sequential([
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(32, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(16, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(5, activation="softmax")
])
model.compile(optimizer=optimizer,
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
history = model.fit(x_train, y_train, epochs=20, batch_size=128)

Epoch 1/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4288 - loss: 1.5257
Epoch 2/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7693 - loss: 1.0160
Epoch 3/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7969 - loss: 0.9320
Epoch 4/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7986 - loss: 0.8964
Epoch 5/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8008 - loss: 0.8398
Epoch 6/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7971 - loss: 0.8331
Epoch 7/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8041 - loss: 0.7904
Epoch 8/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8001 - loss: 0.8020
Epoch 9/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8004 - loss: 0.7929
Epoch 10/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7963 - loss: 0.7917
Epoch 11/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8047 - loss: 0.7725
Epoch 12/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

In [39]:
val_loss, val_accuracy = model.evaluate(x_val, y_val)

129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8114 - loss: 0.7083
